In [1]:
import pandas as pd
import os

In [2]:
file_dir = os.path.join('train_data')

file_path = os.path.join(file_dir, 'merged_data.csv')

df = pd.read_csv(file_path)

print(df)

      Molecule ChEMBL ID Standard Type Standard Relation  Standard Value  \
0          CHEMBL4443947          IC50               '='           0.022   
1          CHEMBL4556091          IC50               '='           0.026   
2          CHEMBL4566431          IC50               '='           0.078   
3          CHEMBL4545898          IC50               '='           0.081   
4          CHEMBL4448950          IC50               '='           0.099   
...                  ...           ...               ...             ...   
74406      CHEMBL5072185          IC50               '='           5.400   
74407      CHEMBL3736012          IC50               '<'          60.000   
74408      CHEMBL5275231          IC50               '='          35.000   
74409      CHEMBL4126054          IC50               '='        6500.000   
74410      CHEMBL3108871          IC50               '='          24.000   

      Standard Units  pChEMBL Value Assay ChEMBL ID Target ChEMBL ID  \
0              

In [3]:
df = df[df['Standard Units'] == 'nM'] # nM이 아닌 행 제거
df = df[df['Standard Type'] == 'IC50'] # nM이 아닌 행 제거
df = df[df['Target Organism'] == 'Homo sapiens'] # Homo sapines가 아닌 행 제거
df = df[df['Standard Relation'] == "'='"] # '='이 아닌 행 제거
df = df[df['Target Type'] == 'SINGLE PROTEIN'] # 'SINGLE PROTEIN'이 아닌 행 제거
df = df.dropna(subset=['Standard Value']) # Standard Value 비어있는 행 제거
df = df.dropna(subset=['Smiles']) # Smiles 비어있는 행 제거

# 인덱스 재설정
df.reset_index(drop=True, inplace=True)

print(df.shape)
print(df)

(57650, 15)
      Molecule ChEMBL ID Standard Type Standard Relation  Standard Value  \
0          CHEMBL4443947          IC50               '='           0.022   
1          CHEMBL4556091          IC50               '='           0.026   
2          CHEMBL4566431          IC50               '='           0.078   
3          CHEMBL4545898          IC50               '='           0.081   
4          CHEMBL4448950          IC50               '='           0.099   
...                  ...           ...               ...             ...   
57645      CHEMBL4084811          IC50               '='         430.000   
57646      CHEMBL5072185          IC50               '='           5.400   
57647      CHEMBL5275231          IC50               '='          35.000   
57648      CHEMBL4126054          IC50               '='        6500.000   
57649      CHEMBL3108871          IC50               '='          24.000   

      Standard Units  pChEMBL Value Assay ChEMBL ID Target ChEMBL ID  \
0  

In [ ]:
# Smiles 열에서 중복된 원소값 확인
smiles_duplicates = df['Smiles'].duplicated(keep=False)  # 모든 중복된 값 표시
duplicated_smiles_df = df[smiles_duplicates]

# 중복된 Smiles 확인
if not duplicated_smiles_df.empty:
    print("중복된 Smiles 값들:")
    print(duplicated_smiles_df)
else:
    print("Smiles 열에 중복된 값이 없습니다.")


# Molecule ChEMBL ID 열에서 중복된 원소값 확인
chembl_id_duplicates = df['Molecule ChEMBL ID'].duplicated(keep=False)  # 모든 중복된 값 표시
duplicated_chembl_id_df = df[chembl_id_duplicates]

# 중복된 Molecule ChEMBL ID 확인
if not duplicated_chembl_id_df.empty:
    print("중복된 Molecule ChEMBL ID 값들:")
    print(duplicated_chembl_id_df)
else:
    print("Molecule ChEMBL ID 열에 중복된 값이 없습니다.")


# 3. Smiles와 Target Name 열에서 행의 중복 확인 및 제거
smiles_target_duplicates = df.duplicated(subset=['Smiles', 'Target Name'], keep=False)
duplicated_smiles_target_df = df[smiles_target_duplicates]

if not duplicated_smiles_target_df.empty:
    print("Smiles와 Target Name 열에 중복된 행들:")
    print(duplicated_smiles_target_df)
    # 중복된 행 제거 (첫 번째만 남기고 나머지 중복된 행 제거)
    df = df.drop_duplicates(subset=['Smiles', 'Target Name'], keep='first')
    print("중복된 Smiles와 Target Name 행 제거 후 데이터프레임:")
    print(df)
else:
    print("Smiles와 Target Name 열에 중복된 행이 없습니다.")


In [4]:
# 제거할 열 목록
columns_to_drop = ['Molecule ChEMBL ID','Standard Type', 'Standard Relation', 'Standard Units', 
                   'pChEMBL Value','Assay ChEMBL ID', 'Target ChEMBL ID', 'Target Organism', 'Target Type',
                   'Document ChEMBL ID', 'IC50_nM', 'pIC50']

# 지정된 열 제거
df = df.drop(columns=columns_to_drop) # columns_to_drop이 아닌 행 거거

print(df.shape)
print(df)


(57650, 3)
       Standard Value                                 Target Name  \
0               0.022  Interleukin-1 receptor-associated kinase 4   
1               0.026  Interleukin-1 receptor-associated kinase 4   
2               0.078  Interleukin-1 receptor-associated kinase 4   
3               0.081  Interleukin-1 receptor-associated kinase 4   
4               0.099  Interleukin-1 receptor-associated kinase 4   
...               ...                                         ...   
57645         430.000                                 Tankyrase-2   
57646           5.400                                 Tankyrase-2   
57647          35.000                                 Tankyrase-2   
57648        6500.000                                 Tankyrase-2   
57649          24.000                                 Tankyrase-2   

                                                  Smiles  
0      CN[C@@H](C)C(=O)N[C@H](C(=O)N1C[C@@H](NC(=O)CC...  
1      CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cn

In [5]:
new_column_order = ['Smiles', 'Target Name', 'Standard Value']
df = df[new_column_order]

# 빈 열 추가
df['Smiles_feature_vector'] = pd.Series([None] * len(df))
df['image_feature_vector'] = pd.Series([None] * len(df))
df['target_protein_vector'] = pd.Series([None] * len(df))

# 결과 확인
print(df)

                                                  Smiles  \
0      CN[C@@H](C)C(=O)N[C@H](C(=O)N1C[C@@H](NC(=O)CC...   
1      CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...   
2      CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...   
3      CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...   
4      COc1cc2c(OC[C@@H]3CCC(=O)N3)ncc(C#CCCCCCCCCCCC...   
...                                                  ...   
57645  O=c1[nH]c2ccccc2n1[C@H]1CC[C@H](c2nnc(-c3ccncn...   
57646  CCOc1ccc(-c2nnc([C@H]3C[C@H](NC(=O)c4ccnc5cccn...   
57647            CC(C)(C)c1ccc(-c2cn3cncc3c(=O)[nH]2)cc1   
57648                  NC(=O)c1ccc(Oc2cccc(C(N)=O)c2)cc1   
57649                           O=c1[nH]c2sccc2c2ccccc12   

                                      Target Name  Standard Value  
0      Interleukin-1 receptor-associated kinase 4           0.022  
1      Interleukin-1 receptor-associated kinase 4           0.026  
2      Interleukin-1 receptor-associated kinase 4           0.078  
3      

In [6]:
df.to_csv('train_data/train_data.csv', index = False)